In [49]:
import pandas as pd

In [50]:
rsp_df = pd.read_csv("../Data/processed/dataset_2016-19")

/tmp/ipykernel_11996/339842733.py:1: DtypeWarning: Columns (8,34,52) have mixed types. Specify dtype option on import or set low_memory=False.
  rsp_df = pd.read_csv("../Data/processed/dataset_2016-19")


In [51]:
rsp_df.iloc[:, [8, 34, 52]].head()  # Need to double-check these columns

,qn1jyear,qn18dyear,qn27byear
0,2016 or later,NaN,2017.0
1,2016 or later,2016.0,NaN
2,2015.0,NaN,2016.0
3,2016 or later,2016.0,2017.0
4,2016 or later,2016.0,NaN


In [57]:
# Create target variable


def create_target(x):
    """Creates a target variable based on if resettlement was fully successful or not"""

    # CRITERIA:
    # Subject is either employed or in education / training, unless they are not eligible
    # Subject makes an income above the poverty level, if employed
    # Subject is either a permanent legal resident or is planning to adjust status

    poverty_thresholds = [11880, 16020, 20160, 24300, 28440]  # 2016 values

    # Check that subject is not unemployed
    if x["ui_emprate"] in ["unemployed", "don't know and/or refused"]:
        return 0

    # If employed, check that target makes wage above poverty threshold for given household size
    if x["ui_emprate"] == "employed":
        try:
            if (
                float(x["ui_qn8a_annual"]) + float(x["ui_qn10a_annual"])
                < poverty_thresholds[int(x["numppl"] - 1)]
            ):
                return 0
        except ValueError:
            return 0

    # Check subject's residency status
    if x["ui_lpr"] not in [
        "already adjusted lpr status",
        "plans to adjust lpr status in future",
    ]:
        return 0

    return 1


rsp_df["t_resettlement"] = rsp_df.apply(lambda x: create_target(x), axis=1)

In [58]:
# Show breakdown of target variable counts
rsp_df["t_resettlement"].value_counts()

t_resettlement
1    2743
0    1792
Name: count, dtype: int64

In [59]:
# Write output

rsp_df.to_csv("../Data/processed/dataset_2016-19_target", index=False)